In [22]:
from datetime import datetime
from time import sleep
import pymysql
import pandas as pd
import threading
import requests
import json
import sys

In [11]:
# MySQL Connection 연결 (DB 연결)
def setting_db():
    conn = pymysql.connect(host="localhost", 
                       port=3306, user="root", 
                       passwd="qlqtmajrwk",
                       db="yejinyDB",
                       charset="utf8")
    cursor = conn.cursor()# (pymysql.cursors.DictCursor) # 리스트로 받기(tuple은 conn.cursor())
    return conn, cursor

In [42]:
def get_now(crp_cd):
    URL = 'http://polling.finance.naver.com/api/realtime.nhn?query=SERVICE_ITEM:'+crp_cd
    data = {'outer': {'inner': 'value'}}
    res = requests.post(URL, data=json.dumps(data))
    tmp = res.json()['result']['areas'][0]['datas'][0]
    try:
        result = {'open':tmp['ov'], 'high':tmp['hv'], 'low':tmp['lv'], 'close':tmp['nv'], 'diff':tmp['nv']-tmp['sv'], 'volume':tmp['aq']}
    except:
        result = {'open':-1, 'high':-1, 'low':-1, 'close':-1, 'diff':-1, 'volume':-1}

    return result

In [12]:
#select crp_cd from Crp table
conn, cursor = setting_db()
sql = "select crp_cd from Corp where crp_cls = 'Y' or crp_cls = 'K';"
cursor.execute(sql)
rows = cursor.fetchall()
conn.commit()
conn.close()

In [14]:
corp_list = []
for i in range(0, len(rows)):
    corp_list.append(rows[i][0])

In [30]:
len(corp_list)

2050

In [33]:
get_now(corp_list[0])

{'close': 10600,
 'diff': -350,
 'high': 11000,
 'low': 10600,
 'open': 11000,
 'volume': 96685}

In [46]:
def daily_Close_update():
    conn, cursor = setting_db()
    new = []
    now_dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    for i in range(0, len(corp_list)):
        tmp = get_now(corp_list[i])
        new.append([corp_list[i], now_dt, tmp['open'], tmp['high'], tmp['low'], tmp['close'], tmp['diff'], tmp['volume']])

    sql = '''insert into Close(
            crp_cd, today_dt, open, high, low, close, diff, volume)
            values (%s, %s, %s, %s, %s, %s, %s, %s)'''

    cursor.executemany(sql, new)
    conn.commit()
    print(now_dt + "의 Close 데이터가 append 되었습니다.")
    conn.close()

In [ ]:
while True:
    daily_Close_update()
    sleep(72000)

2018-07-22 21:00:51의 Close 데이터가 append 되었습니다.
